In [1]:
from spacy.lang.en import English
from spacy.vocab import Vocab
from transformers import AutoModelWithLMHead, AutoTokenizer
import spacy
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch



## Identifying OOV

In [8]:
class oovChecker():
    
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser"]) # using default tokeniser with NER
        with open('./uncased_L-4_H-512_A-8/vocab.txt') as f:
            # if want to remove '[unusedXX]' from vocab
            # words = [line.rstrip() for line in f if not line.startswith('[unused')]
            words = [line.rstrip() for line in f]
        self.vocab = Vocab(strings=words)
        self.BertTokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
        self.BertModel = AutoModelWithLMHead.from_pretrained("bert-base-cased")
        self.mask = self.BertTokenizer.mask_token
            
        # This is temp
        self.query = "this is my India. My name is Rajat Goel. I use tru. income was $9.4 million compared to the prior year of $2.7 milion. number is 62722.2"

  
    ## query --> "aa bb cc..."
    def misspellIdentify(self, query=''):
        """
        At present, All the following criteria should be met for word to be misspelled
        1. Should not in our vocab
        2. should not be a Person
        3. Should not be a number
        """
        if query == '':
            query = self.query
            
        doc = self.nlp(query)
        misspell = []
        for token in doc:
            if((token.text.lower() not in self.vocab) and 
               (token.ent_type_ != 'PERSON') and 
               (not token.like_num)):
                
                misspell.append(token)
        
        print(misspell)
        return misspell
    
    def candidateGenerator(self, misspellings, query=''):
            
        for token in misspellings:
            if query == '':
                updatedQuery = self.query
            updatedQuery = updatedQuery.replace(token.text, self.mask)
            print(updatedQuery)

            model_input = self.BertTokenizer.encode(updatedQuery, return_tensors="pt")
            mask_token_index = torch.where(model_input == self.BertTokenizer.mask_token_id)[1]
            token_logits = self.BertModel(model_input)[0]
            mask_token_logits = token_logits[0, mask_token_index, :]

            top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()  
            for candidate in top_5_tokens:
                print(updatedQuery.replace(self.mask, self.BertTokenizer.decode([candidate])))
                
            print('=='*20)


        return {"misspell-1":["candidate-1","candidate-2"]}
    
    def candidateRanking(self, misspellingsDict):
        return {"misspell-1":"Best_candidate"}
        
        

In [9]:
checker = oovChecker()
misspellTokens = checker.misspellIdentify()
print('misspellTokens',misspellTokens)
candidate = checker.candidateGenerator(misspellTokens)


[tru, milion]
misspellTokens [tru, milion]
this is my India. My name is Rajat Goel. I use [MASK]. income was $9.4 million compared to the prior year of $2.7 milion. number is 62722.2
this is my India. My name is Rajat Goel. I use it. income was $9.4 million compared to the prior year of $2.7 milion. number is 62722.2
this is my India. My name is Rajat Goel. I use you. income was $9.4 million compared to the prior year of $2.7 milion. number is 62722.2
this is my India. My name is Rajat Goel. I use English. income was $9.4 million compared to the prior year of $2.7 milion. number is 62722.2
this is my India. My name is Rajat Goel. I use this. income was $9.4 million compared to the prior year of $2.7 milion. number is 62722.2
this is my India. My name is Rajat Goel. I use money. income was $9.4 million compared to the prior year of $2.7 milion. number is 62722.2
this is my India. My name is Rajat Goel. I use tru. income was $9.4 million compared to the prior year of $2.7 [MASK]. number 

## OLD CODE
***

In [ ]:
sample2




In [ ]:
from transformers import pipeline
nlp = pipeline("fill-mask")
print(nlp.tokenizer.mask_token)
sample2= 'this is my India. My name is Rajat Goel. I use thru. Net income was $9.4 million compared to the prior year of $2.7 <mask>. my number is 5362722.2'
print(nlp(sample2))

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelWithLMHead.from_pretrained("bert-base-cased")

sequence = 'this is my India. My name is Rajat Goel. It was [MASK]. Net income was $9.4 million compared to the prior year of $2.7 [MASK]. my number is 5362722.2'

print('mask',tokenizer.mask_token)

input = tokenizer.encode(sequence, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
print('mask_token_index',mask_token_index)

token_logits = model(input)[0]
print(token_logits)
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[1].tolist()
print('top_5_tokens',torch.topk(mask_token_logits, 5, dim=1))

for token in top_5_tokens:
    print('token', tokenizer.decode([token]))
#     print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))




In [ ]:
a=[17.1059, 14.7497, 11.3313, 10.9787, 10.6195]
for i in a:
    print(i/sum(a))

In [ ]:
len(nlp.vocab)

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser"])
# nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.Defaults.create_tokenizer(nlp)

doc = nlp("this is my pakistan. My name is Rajat Goel. I use tru. Net income was $9.4 million compared to the prior year of $2.7 milion. my number is 5362722.2")

        
from spacy.vocab import Vocab
with open('./uncased_L-4_H-512_A-8/vocab.txt') as f:
    # if want to remove '[unusedXX]' from vocab
    # words = [line.rstrip() for line in f if not line.startswith('[unused')]
    words = [line.rstrip() for line in f]
    
vocab = Vocab(strings=words)
# for i in tokens:
#     print(i.text.lower() in vocab, i.text)


# a= [token for token in doc if (token.text.lower() not in vocab  & token.ent_type != 'PERSON') ]

lst = []
for token in doc:
    print(token.text, token.ent_type_)
    if((token.text.lower() not in vocab) and (token.ent_type_ != 'PERSON') and (not token.like_num)):
#         print(token.text, token.ent_type_, token.is_digit, token.like_num)
        lst.append(token)
        
print(lst)
# print(a[0].i,a[0], a[0].ent_type)

# print([(ent.text, ent.label_) for ent in doc.ents])
print(nlp.entity.labels)
print(len(vocab))
"apple" in vocab

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
spacy.__version__

In [ ]:
import sys
print(sys.executable)